In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
from pandas import *
import os
import re

In [2]:
def read_job(send_job):
    job_cd = send_job[0]
    job_keyword = send_job[1]
#     job_keyword = send_job
    all_data = []
    data_job = []
    page = 1
    data_comps = []
    index = 0
    i = 1
    data=[]
    while page < 6: ### Searching จำนวน 5 Page ###
        data_comps.clear()
        url = requests.get("https://www.jobthai.com/th/jobs?keyword="
                           +job_keyword+"&salarymin=10000&page="
                           +str(page)
                          )
#         print("Searching Job : "+job_keyword+" Page : "+str(page))
        page = int(page)+1
        data_comps.clear() #ลบข้อมลู Company ออก เพื่อเตรียมบันทึกข้อมูล Company สำหรับหน้าถัดไป
        soup = BeautifulSoup(url.text, 'html.parser')
        count_job = soup.findAll("h2", {"class": "ohgq7e-0 frNqfE"})
        count_company = soup.findAll("span", {"class": "msklqa-2 bLKwZm"})
        count_all_position = soup.find("span", {"id": "count-position"}).text
        count_position = re.findall("[0-9]+", count_all_position)
        
        #### จัดการข้อมูลในส่วน Company เนื่องจาก class ของ Companny มีซ้ำกัน ####
        
        for x in range(len(count_company)):
            comps = soup.findAll("span", {"class": "msklqa-2 bLKwZm"})[x]
            if comps.find('h2') == None:
                count_company = comps.find('h2')
            else:
                data_comps.append(comps.string)    
                
        count_company = 0
        for row in range(len(count_job)): #### นำข้อมูลที่จะเอาไปใช้แปลงเป็น String ####
            job_name = soup.findAll("h2", {"class": "ohgq7e-0 frNqfE"})[row].string
            company = data_comps[count_company]
            count_company = count_company+1
            location = soup.findAll(["span","h3"], {"id": "location-text"})[row].string
            get_href = soup.findAll("a", {"class":"msklqa-11 gmzxCR"})[row]
            url = "jobthai.com"+get_href.get("href")

            ##### Regular Expression [Salary] #####
            
            raw_salary = soup.findAll("span", {"id": "salary-text"})[row].string
            get_num_salary = re.findall("[0-9]+", raw_salary)
            get_num_salary = [i for i  in get_num_salary if i != '00']
#             print(get_num_salary)
            ### ข้อมูล Salary บางบริษัทใส่ไม่เหมือนกันเช่น 25,000-30,000 และ 25k-30k  ###
            
            if len(get_num_salary) <= 2 :
                if '000' in get_num_salary :
                    result_max_salary = '-'
                    result_min_salary = ','.join(get_num_salary[:1])
                    min_salary = result_min_salary+',000'
                else :
                    result_max_salary = ','.join(get_num_salary[1:])
                    result_min_salary = ','.join(get_num_salary[:1])
                    min_salary = result_min_salary+',000'
                    max_salary = result_max_salary
                
            elif len(get_num_salary) == 6 :
                result_min_salary = ','.join(get_num_salary[:1])
                result_max_salary = ','.join(get_num_salary[2:])

            elif len(get_num_salary) >= 7 :
                get_num_salary = get_num_salary[:4]
                result_min_salary = ','.join(get_num_salary[:2])
                result_max_salary = ','.join(get_num_salary[2:])
#                 
            else : 
                result_min_salary = ','.join(get_num_salary[:2])
                result_max_salary = ','.join(get_num_salary[2:])

            ### ถ้าข้อมูลเป็นดังตัวอย่าง >> 25k-30k ###
#             print(result_min_salary,result_max_salary)
            
            if len(result_min_salary) <= 4:
                result_max_salary = result_max_salary
                result_min_salary = result_min_salary[:3]
#                 print(result_min_salary)
                min_salary = result_min_salary+',000'
                if '-' in result_max_salary :
                    max_salary = result_max_salary
                else :
                    max_salary = result_max_salary+',000'
            ### ถ้าข้อมูลเป็นดังตัวอย่าง >> 25,000-30,000 ###    
            elif len(result_min_salary) == 1:
                min_salary = result_min_salary+',000'
            else :
                min_salary = result_min_salary 
                max_salary = result_max_salary
            
            if len(max_salary) >= 7 :
                max_salary = max_salary[:6]
            elif max_salary == ',000':
                max_salary == '-'
                
            index += 1
            data_job = [job_cd,job_keyword,index,job_name,company,location,min_salary,max_salary,url] 
            ### Save Data แต่ละแถวลง data_job [list]
            all_data.append(data_job) ### นำ Data (data_job [list]) แต่ละแถว ลงใน all_data [list] ###
    return all_data

In [3]:
def read_count_position(send_job):
    all_count_position = []
    job_count_position = []
    job_cd = send_job[0]
    job_keyword = [send_job[1]]

    now = datetime.now()
    dt_string = now.strftime('%d/%m/%Y %H:%M:%S')
    
    for job in job_keyword :
        url = requests.get("https://www.jobthai.com/th/jobs?keyword="+job+"&salarymin=10000")
        soup = BeautifulSoup(url.text, 'html.parser')
        count_all_position = soup.find("span", {"id": "count-position"}).text
        if len(count_all_position) >= 21:
            count_position = re.findall(r'\d+,\d+', count_all_position)
            print("Job : "+job+ ", Number of all Job : "+count_position[0])
            job_count_position += job_keyword[0], count_position[0], dt_string
        else :
            count_position = re.findall(r'\d+', count_all_position)
            print("Job : "+job+ ", Number of all Job : "+count_position[0])
            job_count_position += job_cd, job_keyword[0], count_position[0],dt_string
            
        all_count_position = job_count_position
    return all_count_position
    

In [4]:
data_ISCO = read_csv('ISCO_08_thai - Sheet1.csv')
kw_job = data_ISCO['TITLE'].tolist()
code_job = data_ISCO['CODE'].tolist()
all_list_keyword = []

for number in range(len(kw_job)):
    all_list_keyword.append([str(code_job[number]),str(kw_job[number])]) 


In [5]:
# test_dict = {}
# for number in range(len(kw_job)):
#     test_dict[code_job[number]] = kw_job[number]
# #     all_list_keyword.append([code_job[number],kw_job[number]])

In [ ]:
##test
jobb = all_list_keyword[:]
# jobb = [['1023','ผู้จัดการด้านการเงิน']]
job_data = []
position_data = []
for sj in jobb:
#     print(sj)
    position_data.append(read_count_position(sj))
    job_data.append(read_job(sj))

Job : ทหารชั้นสัญญาบัตร, Number of all Job : 0
Job : ทหารชั้นประทวน, Number of all Job : 12
Job : ทหารยศอื่นๆ , Number of all Job : 0
Job : ผู้บัญญัติกฎหมาย, Number of all Job : 1
Job : ข้าราชการระดับอาวุโส, Number of all Job : 0
Job : ผู้นำตามประเพณีและหัวหน้าหมู่บ้าน, Number of all Job : 0
Job : เจ้าหน้าที่ระดับอาวุโสขององค์การที่ดำเนินการเพื่อผลประโยชน์เฉพาะอย่าง, Number of all Job : 0
Job : กรรมการผู้จัดการและผู้บริหารระดับสูง, Number of all Job : 3
Job : ผู้จัดการด้านการเงิน, Number of all Job : 132
Job : ผู้จัดการด้านทรัพยากรบุคคล, Number of all Job : 28
Job : ผู้จัดการด้านนโยบายและแผน, Number of all Job : 17
Job : ผู้จัดการด้านการบริการธุรกิจและการบริหารจัดการ  ซึ่งมิได้จัดประเภทไว้ในที่อื่น, Number of all Job : 0
Job : ผู้จัดการด้านการขายและการตลาด, Number of all Job : 71
Job : ผู้จัดการด้านโฆษณาและประชาสัมพันธ์, Number of all Job : 7
Job : ผู้จัดการด้านวิจัยและพัฒนา, Number of all Job : 16
Job : ผู้จัดการด้านการผลิตในภาคการเกษตรและการป่าไม้, Number of all Job : 0
Job : ผู้จัดก

Job : ช่างเทคนิคด้านเหมืองแร่และโลหะวิทยา, Number of all Job : 0
Job : ช่างเขียนแบบ, Number of all Job : 65
Job : ช่างเทคนิคด้านวิทยาศาสตร์กายภาพและวิศวกรรมศาสตร์  ซึ่งมิได้จัดประเภทไว้ในที่อื่น, Number of all Job : 0
Job : หัวหน้าคุมงานด้านเหมืองแร่, Number of all Job : 0
Job : หัวหน้าคุมงานด้านการผลิต, Number of all Job : 2
Job : หัวหน้าคุมงานด้านการก่อสร้าง, Number of all Job : 4
Job : ช่างเทคนิคควบคุมเครื่องจักรโรงงานผลิตพลังงาน, Number of all Job : 0
Job : ช่างเทคนิคควบคุมเครื่องจักรไอน้ำและหม้อน้ำ, Number of all Job : 0
Job : ช่างเทคนิคควบคุมเครื่องจักรโรงงานแปรรูปทางเคมี, Number of all Job : 0
Job : ช่างเทคนิคควบคุมเครื่องจักรโรงงานกลั่นปิโตรเลียมและก๊าซธรรมชาติ, Number of all Job : 0
Job : ช่างเทคนิคควบคุมกระบวนการผลิตโลหะ, Number of all Job : 0
Job : ช่างเทคนิคควบคุมกระบวนการอื่นๆ  ซึ่งมิได้จัดประเภทไว้ในที่อื่น, Number of all Job : 0
Job : เจ้าหน้าที่เทคนิคด้านวิทยาศาสตร์สิ่งมีชีวิต  (ยกเว้นแพทย์), Number of all Job : 0
Job : เจ้าหน้าที่เทคนิคด้านการเกษตรและการประมง, Number o

Job : เพื่อนร่วมกิจกรรมและผู้ดูแลของใช้ส่วนตัว, Number of all Job : 0
Job : สัปเหร่อและเจ้าหน้าที่ฉีดยาศพ, Number of all Job : 0
Job : ช่างตัดแต่งขนสัตว์เลี้ยงและพนักงานดูแลสัตว์เลี้ยง, Number of all Job : 0
Job : ครูสอนขับรถ, Number of all Job : 1
Job : พนักงานบริการส่วนบุคคล  ซึ่งมิได้จัดประเภทไว้ในที่อื่น, Number of all Job : 0
Job : ผู้จำหน่ายสินค้าตามแผงลอยและตลาด, Number of all Job : 0
Job : ผู้จำหน่ายอาหารตามถนน, Number of all Job : 0
Job : เจ้าของร้านค้า, Number of all Job : 75
Job : หัวหน้าคุมงานในร้านค้า, Number of all Job : 11
Job : พนักงานช่วยขายในร้านค้า, Number of all Job : 36
Job : พนักงานแคชเชียร์และเสมียนขายตั๋ว, Number of all Job : 0
Job : นายแบบและนางแบบแฟชั่น  และผู้แสดงแบบอื่นๆ, Number of all Job : 0
Job : พนักงานสาธิตสินค้า, Number of all Job : 12
Job : ผู้จำหน่ายสินค้าตามบ้าน, Number of all Job : 12
Job : พนักงานขายในศูนย์บริการลูกค้า, Number of all Job : 10
Job : พนักงานบริการในสถานีบริการน้ำมันเชื้อเพลิง, Number of all Job : 0
Job : พนักงานเคาน์เตอร์บริการอาหาร

In [ ]:
df_job_count = pd.DataFrame(position_data)
df_job_count = df_job_count.rename(columns={0:'Code', 1:'Searched keyword', 2:'Number of all Job', 3:'Date time'})
df_job_count

In [ ]:
lst_all = []
for job_row in job_data:
    for occu in job_row:
        lst_all.append(occu)

In [ ]:
df = pd.DataFrame(lst_all)
df = df.rename(columns={0:'Code', 1: 'Searched keyword' , 2: 'Index', 3: 'Job title', 4: 'Company', 5:'Location', 6:'Min salary', 7:'Max salary', 8:'Link'})
df


In [ ]:
os.makedirs('csv', exist_ok=True)  
df.to_csv('csv/job_salary.csv' , index=False ,encoding="utf-8-sig")

In [ ]:
os.makedirs('csv', exist_ok=True)  
df_job_count.to_csv('csv/count_position_job.csv' , index=False ,encoding="utf-8-sig")